In [2]:
%pip install vosk
%pip install pyaudio
%pip install transformers
%pip install coqui_tts

Note: you may need to restart the kernel to use updated packages.


^C
Note: you may need to restart the kernel to use updated packages.


     ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
     ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
     -- ------------------------------------- 0.1/1.4 MB 653.6 kB/s eta 0:00:03
     --------------- ------------------------ 0.6/1.4 MB 3.9 MB/s eta 0:00:01
     ---------------------------------- ----- 1.2/1.4 MB 6.5 MB/s eta 0:00:01
     ---------------------------------------- 1.4/1.4 MB 6.5 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ---------------------------------------- 0.0/73.5 kB ? eta -:--:--
     -------------------------------

  error: subprocess-exited-with-error
  
  × Building wheel for coqui_tts (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [423 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build\lib.win-amd64-cpython-312
      creating build\lib.win-amd64-cpython-312\TTS
      copying TTS\api.py -> build\lib.win-amd64-cpython-312\TTS
      copying TTS\model.py -> build\lib.win-amd64-cpython-312\TTS
      copying TTS\__init__.py -> build\lib.win-amd64-cpython-312\TTS
      creating build\lib.win-amd64-cpython-312\TTS\bin
      copying TTS\bin\collect_env_info.py -> build\lib.win-amd64-cpython-312\TTS\bin
      copying TTS\bin\compute_attention_masks.py -> build\lib.win-amd64-cpython-312\TTS\bin
      copying TTS\bin\compute_embeddings.py -> build\lib.win-amd64-cpython-312\TTS\bin
      copying TTS\bin\compute_statistics.py -> build\lib.win-amd64-cpython-312\TTS\bin
      copying TTS\bin\eval_encoder.py -> buil

In [4]:
import json
import os
from vosk import Model, KaldiRecognizer
import pyaudio
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from coqui_tts import TTS

# Step 1: Speech Recognition with Vosk
model = Model("vosk-model-cn")
recognizer = KaldiRecognizer(model, 16000)

p = pyaudio.PyAudio()
stream = p.open(format=pyaudio.paInt16, channels=1, rate=16000, input=True, frames_per_buffer=8000)
stream.start_stream()

print("Speak now...")
while True:
    data = stream.read(4000, exception_on_overflow=False)
    if recognizer.AcceptWaveform(data):
        result = json.loads(recognizer.Result())
        chinese_text = result['text']
        break

stream.stop_stream()
stream.close()
p.terminate()

print(f"You said: {chinese_text}")

# Step 2: Translate to English (Assuming using a pre-trained model or local translation tool)
# Placeholder for translation step
translated_text = chinese_text  # Replace with actual translation code

# Step 3: Generate reply using GPT-2
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

input_ids = tokenizer.encode(translated_text, return_tensors='pt')
output = model.generate(input_ids, max_length=150)
reply_text = tokenizer.decode(output[0], skip_special_tokens=True)

print(f"Generated reply: {reply_text}")

# Step 4: Translate reply back to Chinese (Assuming using a pre-trained model or local translation tool)
# Placeholder for translation step
translated_reply = reply_text  # Replace with actual translation code

# Step 5: Text-to-Speech with Mozilla TTS
tts = TTS(model_name="tts_models/zh-CN/baker/tacotron2-DDC", progress_bar=True, gpu=False)
tts.tts_to_file(text=translated_reply, file_path="reply.wav")

# Play the audio
os.system("aplay reply.wav")


ModuleNotFoundError: No module named 'TTS'